# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.

Для выполнения бонусной части потребуется:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — универсальная модель для обучения различных векторных представлений, разработанная командой Facebook.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*


### Вектора слов/

Для решения вам потребуются две модели векторных представлений слов:

 - [Предобученные векторные представления слов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которые были обучены с помощью стандартной модели word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.
 - Векторные представления слов, полученные с помощью StarSpace на выборке StackOverflow. Вам потребуется обучить эту модель самим во второй части задания.

In [ ]:
# Download Google vectors to directory *target_dir*

from download_utils import download_google_vectors
download_google_vectors(target_dir='.')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [ ]:
import gensim

In [ ]:
wv_embeddings = ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого вы можете запустить функцию *check_embeddings*. Она запускает 3 теста:
1. Находит наиболее похожие слова для заданных "положительных" и "отрицательных" слов.
2. Находит, какое слово из заданного списка не встречается с остальными.
3. Находит наиболее похожее слово для заданного.

In [ ]:
from tests import check_embeddings
print(check_embeddings(wv_embeddings))

### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор. 

<font color='red'>**Реализуйте функцию *question_to_vec_by_mean*, работающую по такой логике. **</font>

In [ ]:
import numpy as np

In [ ]:
def question_to_vec_by_mean(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    ##########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
    ##########################

Для базовой проверки решения запустите клетку ниже.

In [ ]:
from tests import question_to_vec_tests
print(question_to_vec_tests(question_to_vec, wv_embeddings))

Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из *N* вопросов *R* случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели *R + 1* примеров и смотреть на позицию дубликата.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### DCG@K
Второй метрикой будет упрощенная [DCG метрика](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K],$$
где $rank_{dup_i}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$. С такой метрикой модель штрафуется за низкую позицию корректного ответа.

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. Пусть $N = 1$, $R = 3$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

<font color='red'>**Реализуйте функции *hits_count* и *dcg_score*. **</font> 

Каждая функция имеет два аргумента: *dup_ranks* и *k*. *dup_ranks* является списком, который содржит *рейтинги дубликатов* (их позиции в ранжированном списке). Например, *dup_ranks = [2]* для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of ranks of the duplicates; one rank per question; 
                   length is a number of questions that we check (N); 
                   rank is a number from 1 to len(candidates for the question).
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for the current ranking.
    """
    ############################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ############################

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of ranks of the duplicates; one rank per question; 
                   length is a number of questions that we check (N); 
                   rank is a number from 1 to len(candidates for the question).
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for the current ranking.
    """
    ############################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ############################

Протестируйте функции. Успешное прохождение базовых тестов еще не гарантирует корректности реализации!

In [ ]:
from tests import test_hits
print(test_hits(hits_count))

In [ ]:
from tests import test_dcg
print(test_dcg(dcg_score))

### Ранжирование вопросов StackOverflow

Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (test.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую выборку для оценки качества текущего решения.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [ ]:
validation = ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

<font color='red'>**Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния.**</font>
    
Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список *[(2, c), (0, a), (1, b)]*.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    pass

Протестируйте работу функции на примерах ниже.

In [ ]:
from tests import test_rank_candidates
print(test_rank_candidates(rank_candidates, wv_embeddings))

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут).

In [ ]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

Если вы проделали все шаги правильно, то вы должны разочароваться полученными результатами. Давайте попробуем понять, почему качество модели такое низкое. Когда вы работаете с какими-либо данными, очень полезно первым делом посмотреть на них глазами. Выведите несколько вопросов из наших данных:

In [ ]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])
    print()

## Часть 2. Предобработка данных (1 балл)

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

<font color='red'>**Реализуйте функцию предобработки текстов.**</font>

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации на пробелы;
- Удалить "плохие" символы;
- Удалить стопслова.

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
def text_prepare(text):
    """
        text: a string
        
        return: modified string
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################

<font color='red'>**Теперь преобразуйте все вопросы из тестовой выборки. Оцените, как изменилось качество. Сделайте выводы.**</font>

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

## Часть 3. Представления для неизвестных слов. (2 балла)

<font color='red'>**Оцените долю слов в выборке, для которых нет эмбеддинга в модели.**</font>

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм _wh, whe, her, ere, re_

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

<font color='red'>**Реализуйте предложенную модель ниже.**</font>

Используйте класс nn.EmbeddingBag для построения среднего вектора представлений.

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class TrigrammEmbeddingsModel(nn.Module):
    def __init__(self, all_known_tokens, embedding_dim=300):
        """
        all_known_tokens : list of str
        
        embedding_dim : int
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
    
    def forward(self, token):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################


<font color='red'>** Обучите модель. Оцените, как изменилось качество. Сделайте выводы.**</font>

Если вы всё реализовали правильно, качество решения должно вырасти.

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

## Бонусная часть: векторные представления StarSpace (2 балла)

В бонусной части вам предлгается обучить эмбеддинги специально для задачи поиска дубликатов с помощью пакета [StarSpace](https://github.com/facebookresearch/StarSpace). К сожалению, его нельзя запустить на Windows, поэтому в этом случае мы рекоммендуем использовать готовый [docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) с пошаговыми инструкциями или воспользоваться платформой google colab.

Данная модель все еще представляет вопросы с помощью усреднения векторов слов, однако обучается по размеченной выборке пар близких вопросов. Это позволяет обучить вектора, которые лучше подходят для конкретной задачи. Напомним, что в модели word2vec обучение происходят по парам близких слов, и на этапе обучения модель ничего не знает о наших планах по их усреднению в пост-обработке.


### Как выбрать  параметры модели?

Ниже приведены некоторые рекомендации, с которых можно начать свои эксперименты.

- Обучение на парах близких предложений соответствует режиму *trainMode = 3*.
- Используйте метод оптимизации adagrad (параметр *adagrad=True*).
- Установите длину фразы равной 1 (параметр *ngrams*), чтобы получить только вектора слов.
- Не используйте большое количество эпох (5 должно быть достаточно).
- Поэкспериментируйте с несколькими размерностями *dim* (например, от 100 до 300).
- Для сравнения векторов используйте *косинусную меру*.
- Установите *minCount* больше 1 (например, 2), если вы не хотите получить вектора для редко встречающихся слов.
- Параметр *verbose=True* будет показывать вам прогресс процесса обучения.
- Параметр *negSearchLimit* отвечает за число отрицательных примеров, которые используются в обучении, рекомендованное значение 10.
- Для ускорения обучения мы рекоммендуем поставить *шаг обучения (learning rate)* равным 0.05.

<font color='red'>** Обучите вектора StarSpace для униграм на обучающей выборке. Не забудьте использовать предобработанную версию данных. **</font>

Если вы следовали инструкциям правильно, то процесс обучения займет около 1 часа. Размер словаря полученных векторных представлений должен быть порядка 100000 (число строк в полученном файле). 

In [ ]:
##### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ TRAINING HAPPENING HERE #####

Ниже вы можете проверить качество работы вашей модели. Так как обучение происходило для конкретной задачи на размеченных данных, то ожидается, что это решение будет иметь более высокое качество.

In [ ]:
starspace_embeddings = ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

In [ ]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                              k, hits_count(ss_prepared_ranking, k)))

<font color='red'>**Опишите результаты ваших экспериментов, сделайте выводы.**</font>

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################